In [1]:
from lightfm_recomender import LightfmRecomender
import os
%load_ext autoreload
%autoreload 2
new_directory = '/Users/vasevooo/projects/GameBuddy'
os.chdir(new_directory)
import pandas as pd 
import requests
import json
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
from lightfm import LightFM
from scipy.sparse import csr_matrix, save_npz, load_npz
import re
from bs4 import BeautifulSoup

/Users/vasevooo/projects/GameBuddy/steam/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/vasevooo/projects/GameBuddy/steam/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


### обновление предобработанных данных для модели 

In [7]:
games = pd.read_csv('data/game_info_with_scores.csv')
item_dict ={}
games = games[['appid', 'Name']].sort_values('appid').reset_index()
games['appid'] = games['appid'].astype(int)

for i in range(games.shape[0]):
    item_dict[int(games.loc[i,'appid'])] = games.loc[i,'Name']

In [8]:
## сохранение словаря по играм для дальнейшего использования

with open("data/for_models/item_dict.json", "w") as outfile:
    json.dump(item_dict, outfile, indent=4)

### Тест класса LigthFM Recomender

In [72]:
item_dict_path = 'data/for_models/item_dict.json'
gametime_df_path = 'data/final_gametime_df.csv'

lfr = LightfmRecomender(gametime_df_path, item_dict_path)

/Users/vasevooo/projects/GameBuddy/steam/lib/python3.9/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [177]:
steam_url = 'https://steamcommunity.com/profiles/76561198102706863/'

# https://steamcommunity.com/profiles/76561198102706863/
# https://steamcommunity.com/id/momuss/ - Серега
# https://steamcommunity.com/profiles/76561197992623144 - Коля
# https://steamcommunity.com/profiles/76561198156258305/ - Дима
# https://steamcommunity.com/profiles/76561197991678198 - Антон

In [178]:
new_user, steamid = lfr.get_user_games(steam_url)
steamid

76561198102706863

In [179]:
matrix, df = lfr.get_csr_matrix_for_all_users(new_user)

In [182]:
## в приложении поставить 100 эпох. 1 для теста

model = lfr.fit_model(matrix, epochs=100)

In [184]:
lfr.sample_recommendation_user(model, df, steamid, threshold = 0, nrec_items = 10)

User: 76561198102706863
Recommended Items:
1- Age of Empires IV: Anniversary Edition
2- Slay the Spire
3- They Are Billions
4- Total War: WARHAMMER III
5- Call of Duty®: Modern Warfare® 3
6- Forts
7- Age of Empires II: Definitive Edition
8- Legion TD 2 - Multiplayer Tower Defense
9- Frostpunk
10- Dirty Bomb®

 Known Likes:
1- Soulstone Survivors
2- 20 Minutes Till Dawn
3- Brotato
4- Vampire Survivors
5- V Rising
6- Dorfromantik
7- The Planet Crafter
8- Loop Hero
9- Melvor Idle
10- eFootball PES 2021 SEASON UPDATE
11- Rogue Legacy 2
12- DEATHLOOP
13- ELDEN RING
14- Halo Infinite
15- Gunfire Reborn
16- Potion Craft: Alchemist Simulator
17- Sea of Thieves 2023 Edition
18- Teardown
19- Crusader Kings III
20- Gas Station Simulator
21- Skul: The Hero Slayer
22- Hades
23- Medieval Dynasty
24- HUMANKIND™
25- Curse of the Dead Gods
26- Fall Guys
27- Cyberpunk 2077
28- Destiny 2
29- Dota Underlords
30- Going Medieval
31- The Ascent
32- Halo: The Master Chief Collection
33- Mortal Kombat 11
34- A

### cold start

In [23]:
df_cold_start = pd.DataFrame({'game_names': ['Counter-Strike', 'Team Fortress Classic', 
                                             'Day of Defeat', 
                                             'Half-Life: Source', 'Half-Life: Opposing Force'],
                   'rankings': [5, 5, 8, 7, 3]})

In [24]:
df_cold_start.head(2)

,game_names,rankings
0,Counter-Strike,5
1,Team Fortress Classic,5


In [35]:
cold_start_user, steamid_cold = lfr.get_user_games_cold_start(df_cold_start)

In [36]:
matrix_cs, df_cs = lfr.get_csr_matrix_for_all_users(cold_start_user)

In [20]:
## в приложении поставить 100 эпох. 1 для теста
model = lfr.fit_model(matrix_cs, epochs=10)

In [38]:
lfr.sample_recommendation_user(model, df_cs, steamid_cold, threshold = 0, nrec_items = 10)

User: 1234567891010102930
Recommended Items:
1- Counter-Strike: Global Offensive
2- Left 4 Dead 2
3- PAYDAY 2
4- Portal 2
5- The Witcher® 3: Wild Hunt
6- Terraria
7- Half-Life 2
8- Garry's Mod
9- The Elder Scrolls V: Skyrim
10- Dying Light

 Known Likes:
1- Half-Life: Source
2- Day of Defeat
3- Team Fortress Classic
4- Counter-Strike


### Валидация 

In [169]:
from lightfm.cross_validation import random_train_test_split

train_data, test_data = random_train_test_split(matrix, test_percentage=0.2, random_state=42)
train_data.shape, test_data.shape, matrix.shape

((15352, 1785), (15352, 1785), (15352, 1785))

In [170]:
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
train_precision = precision_at_k(model, train_data, k=10, num_threads=1).mean()
train_auc = auc_score(model, train_data, num_threads=4).mean()
test_precision = precision_at_k(model, test_data, k=10, num_threads=1).mean()
test_auc = auc_score(model, test_data, num_threads=4).mean()

In [171]:
train_precision, train_auc, test_precision, test_auc

(0.7595651, 0.90443504, 0.19060513, 0.8790407)